In [20]:
from yargy import Parser, rule, and_, or_
from yargy.predicates import is_capitalized
from yargy.interpretation import fact
from yargy.relations import gnc_relation

In [21]:
Name = fact(
    "Name",
    ["first", "last"]
)

BirthDate = fact(
    "BirthDate",
    ["date"]
)

BirthPlace = fact(
    "BirthPlace",
    ["in_", "place"]
)

Entry = fact(
    "Entry",
    ["name", "birth_date", "birth_place"]
)

# Паттерн для даты

In [22]:
from yargy.predicates import gte, lte
from yargy.predicates import caseless, normalized, dictionary


MONTHS = {
    'январь',
    'февраль',
    'март',
    'апрель',
    'мая',
    'июнь',
    'июль',
    'август',
    'сентябрь',
    'октябрь',
    'ноябрь',
    'декабрь'
}
YEAR_WORDS = or_(
    rule(caseless('г'), '.'),
    rule(normalized('год'))
)
Date = fact(
    'Date',
    ['year', 'month', 'day']
)

DAY = and_(
    gte(1),
    lte(31)
).interpretation(
    Date.day
)
MONTH = and_(
    gte(1),
    lte(12)
).interpretation(
    Date.month
)
YEAR = and_(
    gte(1),
    lte(2025)
).interpretation(
    Date.year
)
MONTH_NAME = dictionary(
    MONTHS
).interpretation(
    Date.month
)
DATE = rule(
    normalized("в").optional(),
    DAY.optional(),
    MONTH_NAME.optional(),
    YEAR,
    YEAR_WORDS.optional()
).interpretation(Date)

# Паттерн для имени

In [23]:
gnc = gnc_relation()

NAME = rule(
    is_capitalized().interpretation(Name.first.inflected()).optional(),
    is_capitalized().interpretation(Name.last.inflected()),
).interpretation(
    Name
)

In [24]:
parser = Parser(NAME)
text = "Борис Васильев родился 21 мая 1924 года в Смоленске."

for match in parser.findall(text):
    print(match.fact)

Name(first='борис', last='васильев')
Name(first=None, last='смоленск')


# Паттерн для места рождения

In [25]:
PLACE = rule(
    rule(normalized("в")).interpretation(BirthPlace.in_),
    rule(is_capitalized()).interpretation(BirthPlace.place),
).interpretation(
    BirthPlace
)

parser = Parser(PLACE)
text = "Борис Васильев родился 21 мая 1924 года в Смоленске."

for match in parser.findall(text):
    print(match.fact)

BirthPlace(in_='в', place='Смоленске')


# Общий паттерн

In [26]:
PlaceDate = fact(
    "PlaceDate",
    ["place", "date"]
)

DatePlace = fact(
    "DatePlace",
    ["date", "place"]
)

RULE = rule(
    NAME.interpretation(Entry.name),
    normalized("родился"),
    or_(
        rule(
            PLACE.interpretation(Entry.birth_place),
            DATE.interpretation(Entry.birth_date)
        ).interpretation(PlaceDate),
        rule(
            DATE.interpretation(Entry.birth_date),
            PLACE.interpretation(Entry.birth_place)
        ).interpretation(DatePlace),
    )
).interpretation(Entry)

parser = Parser(RULE)
text = "Борис Васильев родился 21 мая 1924 года в Смоленске."

for match in parser.findall(text):
    print(match.fact)
    print([_.value for _ in match.tokens])

Entry(name=Name(first='борис', last='васильев'), birth_date=None, birth_place=None)
['Борис', 'Васильев', 'родился', '21', 'мая', '1924', 'года', 'в', 'Смоленске']


In [27]:
PlaceDate = fact(
    "PlaceDate",
    ["place", "date"]
)

DatePlace = fact(
    "DatePlace",
    ["date", "place"]
)


RULE = rule(
    NAME.interpretation(Entry.name),
    caseless("родился"),
    or_(
        rule(
            PLACE.interpretation(Entry.birth_place),
            DATE.interpretation(Entry.birth_date)
        ),
        rule(
            DATE.interpretation(Entry.birth_date),
            PLACE.interpretation(Entry.birth_place)
        ),
    )
).interpretation(Entry)

parser = Parser(RULE)
text = "Борис Васильев родился 21 мая 1924 года в Смоленске."

for match in parser.findall(text):
    print(match.fact.name)
    print(match.fact.birth_date)
    print(match.fact.birth_place)

Name(first='борис', last='васильев')
Date(year='1924', month='мая', day='21')
BirthPlace(in_='в', place='Смоленске')


# Парсинг из всех данных

In [ ]:
import gzip

from dataclasses import dataclass

@dataclass
class Text:
    label: str
    title: str
    text: str


def read_texts(fn):
    with gzip.open(fn, "rt", encoding="utf-8") as f:
        for line in f:
            yield Text(*line.strip().split("\t"))

texts = list(read_texts("./data/news.txt.gz"))

In [29]:
texts[:10]

[Text(label='style', title='Rolex наградит победителей регаты', text='Парусная гонка Giraglia Rolex Cup пройдет в Средиземном море в 64-й раз. Победители соревнования, проводимого с 1953 года Yacht Club Italiano, помимо других призов традиционно получают в подарок часы от швейцарского бренда Rolex. Об этом сообщается в пресс-релизе, поступившем в редакцию «Ленты.ру» в среду, 8 мая. Rolex Yacht-Master 40 Фото: пресс-служба Mercury Соревнования будут проходить с 10 по 18 июня. Первый этап: ночной переход из Сан-Ремо в Сен-Тропе 10-11 июня (дистанция 50 морских миль — около 90 километров). Второй этап: серия прибрежных гонок в бухте Сен-Тропе с 11 по 14 июня. Финальный этап пройдет с 15 по 18 июня: оффшорная гонка по маршруту Сен-Тропе — Генуя (243 морских мили — 450 километров). Маршрут проходит через скалистый остров Джиралья к северу от Корсики и завершается в Генуе.Регата, с 1997 года проходящая при поддержке Rolex, считается одной из самых значительных яхтенных гонок в Средиземноморь

In [30]:
from tqdm import tqdm

matches = []

for text in tqdm(texts):
    for match in parser.findall(text.text):
        print(match.fact)
        matches.append((text, match))

  9%|▉         | 902/10000 [00:35<05:02, 30.04it/s]

Entry(name=Name(first=None, last='трэмиел'), birth_date=Date(year='1928', month=None, day=None), birth_place=BirthPlace(in_='в', place='Польше'))


 12%|█▏        | 1198/10000 [00:49<05:35, 26.20it/s]

Entry(name=Name(first=None, last='он'), birth_date=Date(year='1865', month='сентября', day='11'), birth_place=BirthPlace(in_='в', place='Польше'))


 12%|█▏        | 1231/10000 [00:50<05:24, 27.02it/s]

...

 99%|█████████▊| 9858/10000 [06:26<00:04, 32.45it/s]

Entry(name=Name(first='алексей', last='ремиз'), birth_date=Date(year='1877', month=None, day=None), birth_place=BirthPlace(in_='в', place='Москве'))


 99%|█████████▉| 9898/10000 [06:28<00:03, 27.73it/s]

Entry(name=Name(first='михаил', last='алгаш'), birth_date=Date(year='1988', month=None, day=None), birth_place=BirthPlace(in_='в', place='Омске'))


100%|██████████| 10000/10000 [06:31<00:00, 25.55it/s]


Проверяем

In [31]:
for item in matches:
    text = item[0]
    match = item[1]
    left, right = match.tokens[0].span.start, match.tokens[-1].span.stop
    print(f"Extracted from: {text.text[left:right]}\nmatched: {match.fact}")
    print('-' * 80)

Extracted from: Трэмиел родился в 1928 году в Польше
matched: Entry(name=Name(first=None, last='трэмиел'), birth_date=Date(year='1928', month=None, day=None), birth_place=BirthPlace(in_='в', place='Польше'))
--------------------------------------------------------------------------------
Extracted from: Он родился в Польше 11 сентября 1865 года
matched: Entry(name=Name(first=None, last='он'), birth_date=Date(year='1865', month='сентября', day='11'), birth_place=BirthPlace(in_='в', place='Польше'))
--------------------------------------------------------------------------------
Extracted from: Ребенок родился в Калифорнии 25
matched: Entry(name=Name(first=None, last='ребёнок'), birth_date=Date(year='25', month=None, day=None), birth_place=BirthPlace(in_='в', place='Калифорнии'))
--------------------------------------------------------------------------------
Extracted from: Дмитрий Чернявский родился в Артемовске 5 марта 1992 года
matched: Entry(name=Name(first='дмитрий', last='чернявск